344. Sort Integers by the Number of 1 Bits

https://leetcode.cn/problems/sort-integers-by-the-number-of-1-bits

## 核心考察点

这道题的标签包含“排序”和“位运算”，但其核心考察点并非要求从零实现排序算法或必须使用位运算技巧。题目的主要考察点是：

1.  **子问题的分解能力**：能否将问题拆解为“为每个数字计算其二进制中 1 的数量”和“根据这个计算结果进行自定义排序”两个子问题。
2.  **自定义排序的实现**：是否熟悉所用语言的排序工具，并能用其实现“先按A标准，再按B标准”这样的多级排序逻辑。
3.  **基础编程能力**：能够清晰、准确地实现上述逻辑。

因此，在解题时，可以直接调用语言内置的高效排序函数，重点在于如何为其提供正确的排序规则。

---
## 深入解析：`sorted()` 函数

`sorted()` 是 Python 的一个内置函数，可对任何可迭代对象进行排序。它与 `list.sort()` 方法不同，`sorted()` 总是返回一个**新的、排序后的列表**，而不修改原始对象。

**基本语法**: `sorted(iterable, *, key=None, reverse=False)`

### 1. `iterable` (第一个参数)

`sorted()` 的必需参数，可以是任何可迭代的对象，例如列表、元组、字符串或字典的键。在内部机制上，`sorted()` 在排序前会先将可迭代对象中的所有元素加载到一个临时的列表中，然后对这个临时列表进行排序。

### 2. 排序依据

#### a. 默认比较规则

当不提供 `key` 参数时，`sorted()` 使用元素之间内置的比较规则。该规则依赖于对象的“富比较”魔法方法 (Rich Comparison Magic Methods)，主要通过调用 `__lt__` (`<`) 方法来确定两个元素的相对顺序。

#### b. `key` 参数

`key` 是一个可选的关键字参数，它接收一个函数。此函数会在排序比较前作用于 `iterable` 中的每一个元素，`sorted()` 将根据该函数返回的“键”来进行排序。

`key` 所指定的函数在定义上**只接收一个参数**（即 `iterable` 中的元素），其作用是为该元素生成一个用于比较的“键”或“代理值”。

* **`key=func` (单参数函数)**: `func` 的作用是 **“提取”** 或 **“转换”**。排序算法不直接比较元素 `x` 和 `y`，而是比较 `func(x)` 和 `func(y)` 的结果。

* **双参数比较函数**: 在 Python 3 中，直接定义两个元素如何比较的函数（例如 `compare(a, b)`）不能直接用于 `key` 参数。这种接收两个参数并返回负数、零或正数的比较逻辑，需要通过 `functools.cmp_to_key` 进行转换后才能使用。

### 3. `reverse` 参数

`reverse` 是一个布尔类型的关键字参数，用于控制排序顺序。

* `reverse=False` (默认值): 按升序排列。
* `reverse=True`: 按降序排列。

---

## 如何计算二进制中 “1” 的个数

为每个数字提供排序依据的第一步，是计算其二进制表示中 “1” 的数量。主要有以下几种方法：

### 方法一：内置 `num.bit_count()` 方法 (Python 3.10+, 效率最高)

直接调用整数对象的内置方法，是 Python 中计算比特数的最佳选择。

* **机制**: `num.bit_count()` 在底层用 C 语言实现，并且经过了高度优化。在支持的硬件上，它甚至可以直接转换为一条**硬件指令**（如 `POPCNT` 指令），用一个 CPU 指令就能完成整个计数过程。

### 方法二：库函数法 `bin().count()` (通用且简洁)

利用 Python 内置的函数和字符串方法，代码非常简洁。

* **机制**: 这个操作分为两步：
    1.  **`bin(num)`**: 将整数 `num` 转换为一个以 `"0b"` 开头的二进制字符串。
    2.  **`.count('1')`**: 调用字符串的 `count()` 方法，统计其中字符 `'1'` 出现的次数。

### 方法三：位运算法 (Brian Kernighan 算法)

一种高效的位运算技巧，循环次数等于 “1” 的个数，因此对于 “1” 很稀疏的数字（即 “1” 的数量远少于总位数）效率很高。

```python
def count_bits_kernighan(num: int) -> int:
    count = 0
    while num > 0:
        # 这个操作会精确地将 n 的二进制表示中最右边的那个 1 变成 0
        num = num & (num - 1)
        count += 1
    return count
```

### 方法四：朴素循环法 (基础实现)

通过循环和基本算术运算，手动实现计数逻辑，效率较低。

```python
def count_bits_loop(num: int) -> int:
    count = 0
    while num > 0:
        if num % 2 == 1:  # 检查最低位是否为 1
            count += 1
        num = num // 2    # 效果等同于二进制右移一位
    return count
```

---

## 如何实现自定义排序

### 方案一：利用 `key` 函数返回元组 (推荐)

这是最 Pythonic、最简洁的实现方式。Python 在对元组进行排序时，会自动按元组内元素的顺序进行多级比较。

```python
from typing import List

class Solution:
    def sortByBits(self, arr: List[int]) -> List[int]:
        # 对于每个 num，key 函数返回一个元组 (比特数, num 本身)
        # sorted() 会先按比特数排序，比特数相同时再按 num 排序
        return sorted(arr, key=lambda num: (num.bit_count(), num))
```

### 方案二：利用自定义比较函数 (更通用)

虽然在 Python 中不常用，但理解这种方法有助于了解其他语言（如 C++, Java）的实现方式。Python 中需要借助 `functools` 模块的 `cmp_to_key` 来将一个接收两个参数的“比较函数”转换成可用于 `key` 参数的函数。

比较函数 `compare(a, b)` 的规则：
* 如果 `a` 应该在 `b` 前面，返回负数 (如 -1)。
* 如果 `a` 应该在 `b` 后面，返回正数 (如 1)。
* 如果两者顺序无所谓，返回 0。

```python
from typing import List
from functools import cmp_to_key

class Solution:
    def sortByBits(self, arr: List[int]) -> List[int]:
        # 1. 定义一个接收两个参数的比较函数
        def compare(a: int, b: int) -> int:
            bit_count_a = a.bit_count()
            bit_count_b = b.bit_count()

            # 主标准：比较比特数
            if bit_count_a < bit_count_b:
                return -1
            if bit_count_a > bit_count_b:
                return 1
            
            # 次标准：比特数相等时，比较数值本身
            if a < b:
                return -1
            if a > b:
                return 1
            
            return 0

        # 2. 使用 cmp_to_key 将比较函数转换为 key 函数
        return sorted(arr, key=cmp_to_key(compare))
```
这种方法更冗长，但清晰地展示了多级比较的底层逻辑，这个逻辑在其他语言中是必需的。

---

## 最终复杂度分析

* **时间复杂度**: $O(n \log n)$
    * 无论采用哪种方法，算法的主体都是排序。Python 内置的 `sorted()` 函数（Timsort）的时间复杂度是 $O(n \log n)$，其中 `n` 是数组 `arr` 的长度。
    * 为每个元素计算排序键（`key` 函数）的耗时为 $O(k)$，其中 `k` 是整数的二进制位数（对于题目中的数值范围，`k` 是一个小于 30 的常数）。
    * 因此，总的时间复杂度由排序主导，为 $O(k \cdot n \log n) = O(n \log n)$。

### 空间复杂度分析

#### `sorted()` 函数的空间复杂度: $O(n)$

`sorted()` 函数的空间复杂度主要由三部分构成：**存储返回结果的列表**、**排序前用于缓存键的临时列表**和 **Timsort 算法执行时所需的辅助空间**。

1.  **返回结果的存储空间**:
    `sorted()` 函数的核心功能是创建一个**全新的、排序后的列表**作为返回值。为了存储这个新列表，程序必须分配一个能容纳 `n` 个元素的内存空间。因此，仅这一项就决定了其空间复杂度的下限是 $O(n)$。

2.  **`key` 函数与装饰键的空间使用**:
    为了提高效率（特别是当 `key` 函数计算耗时），`sorted` 函数并**不会**在每次比较时都重新调用 `key` 函数。它采用了一种名为 **“装饰-排序-反装饰” (Decorate-Sort-Undecorate)** 的缓存策略：
    * **装饰 (Decorate)**：在排序开始**之前**，函数会遍历一次原始列表，为每个元素**调用一次 `key` 函数**，并生成一个临时的 `(key, original_value)` 元组。
    * **创建临时列表**：所有这些 `(key, original_value)` 元组会被存储在一个**新的临时列表**中。这个列表的大小为 `n`，因此会占用 $O(n)$ 的额外空间。
    * **排序 (Sort)**：后续的 Timsort 算法将直接对这个包含元组的临时列表进行排序，比较时直接使用元组中已经计算好的 `key`。

    这个机制的**时间优化**效果显著（避免了 `key` 函数的重复计算），但代价是需要一个 $O(n)$ 的额外空间来存储这些“装饰”后的键和值。

3.  **Timsort 算法的辅助空间**:
    除了上述空间外，Timsort 排序算法在执行**过程**中，还需要一些临时的“工作空间”来合并已排序的片段（runs）。这个辅助空间的大小取决于输入数据的有序程度，最优情况下为 $O(\log n)$，最坏情况下为 $O(n)$。

**综合分析**:
`sorted()` 函数的总空间开销是以上三项之和：
$$
T_{\text{空间}} = \underbrace{O(n)}_{\text{最终结果}} + \underbrace{O(n)}_{\text{装饰键值对}} + \underbrace{O(\log n) \sim O(n)}_{\text{Timsort工作空间}}
$$
由于所有项最高都是 $O(n)$，因此在算法分析中，**`sorted()` 函数的总空间复杂度通常记为 $O(n)$**。

## 深入解析：`functools.cmp_to_key`

`functools.cmp_to_key` 是 Python `functools` 模块中的一个工具函数，其作用是将一个接收两个参数的**“比较函数” (comparison function)** 转换为一个接收单一参数的**“键函数” (key function)**。

### 核心目的与背景

Python 3 的内置排序函数（如 `sorted()`）使用 `key` 参数来指定排序规则，该 `key` 函数接收一个参数。这与早期 Python 版本中同时支持 `cmp` 参数（接收一个含两个参数的比较函数）的做法不同。`cmp_to_key` 的目的就是提供一个转换器，使得 `cmp(a, b)` 风格的比较逻辑能够兼容 Python 3 的 `key` 排序体系。

### “富比较”与“贫比较”的由来

#### 旧的“贫比较” (Poor Comparison)
在 Python 2.1 之前，自定义对象的比较逻辑只能通过实现一个 `__cmp__(self, other)` 方法来完成。该方法需要返回负数、零或正数，来分别表示 `self` 小于、等于或大于 `other`。用一个方法处理所有六种比较关系（`<`, `<=`, `==`, `!=`, `>`, `>=`），功能比较单一，因此被称为“贫比较”。

#### 新的“富比较” (Rich Comparison)
从 Python 2.1 开始，引入了一套六个独立的魔法方法来分别处理每一种比较操作。这种设计被认为是“更丰富的 (richer)”，因为它：

1.  **控制更精细**: 开发者可以只定义自己需要的比较行为。例如，一个类可能只关心是否相等 (`__eq__`) 和不相等 (`__ne__`)，而不需要定义大小关系。
2.  **代码更清晰**: `def __lt__(self, other):` 的意图比 `def __cmp__(...): if ... return -1` 要清晰得多。
3.  **可以自动推导**: 通常只需要定义 `__lt__` 和 `__eq__`，Python 就可以自动推导出其他四个比较方法的行为。

这六个富比较方法是：
* `__lt__(self, other)` --- a < b
* `__le__(self, other)` --- a <= b
* `__eq__(self, other)` --- a == b
* `__ne__(self, other)` --- a != b
* `__gt__(self, other)` --- a > b
* `__ge__(self, other)` --- a >= b

`sorted()` 函数在进行默认排序时，会优先使用 `__lt__` 方法来反复比较元素，以确定它们的先后顺序。

### `cmp_to_key` 的工作原理与选择

#### 详细工作机制：代理对象的实现

`sorted()` 函数的核心是比较“键”，而 `cmp_to_key` 的巧妙之处在于，它生成的键函数所返回的“键”本身就是一个**可比较的自定义对象**（代理对象）。

其工作流程如下：

1.  **生成键函数**: 调用 `cmp_to_key(compare)` 会返回一个键函数 `key_func`。
2.  **创建代理对象**: `sorted` 在排序时，会对列表中的每个元素 `x` 调用 `key_func(x)`。此调用会创建一个内部定义的、特殊的“代理”对象，该对象内部存储了对原始元素 `x` 的引用。
3.  **比较代理对象**: 当排序算法需要比较原始列表中的两个元素 `a` 和 `b` 时，它实际上是在比较它们各自对应的代理对象 `proxy_a` 和 `proxy_b`。例如，执行 `proxy_a < proxy_b`。
4.  **调用原始比较函数**: 代理对象 `proxy_a` 的 `__lt__` 方法被预先设定好了。当 `proxy_a < proxy_b` 触发时，`proxy_a` 的 `__lt__` 方法会调用最初传入的 `compare` 函数，并将各自代理的原始元素 `a` 和 `b` 作为参数，即执行 `compare(a, b)`。
5.  **返回比较结果**: `__lt__` 方法根据 `compare(a, b)` 的返回值（-1, 0, 1）来决定自身的返回值（`True` 或 `False`），从而告诉排序算法 `a` 和 `b` 的正确顺序。

本质上，`cmp_to_key` 自动创建了一个轻量级的类，并为其实例（代理对象）实现了所有富比较方法，而这些方法的内部逻辑都指向了用户提供的那个双参数 `compare` 函数。这是一种经典的设计模式，称为**适配器模式**。

#### 如何选择排序实现方式

在 Python 3 中，实现自定义排序逻辑时有以下选择：

1.  **首选：`key` + `lambda` / 普通函数**: 对于多级排序，返回元组 `key=lambda x: (标准1, 标准2)` 的方式最为简洁高效。

2.  **面向对象：实现富比较方法**: 当一个类的实例需要有固有的、全局一致的排序行为时，应当在该类中直接实现 `__lt__` 等富比较方法。

3.  **使用 `cmp_to_key`**: 当排序逻辑复杂，无法用简单的元组表达，并且又不想为此专门创建一个完整的类时，`cmp_to_key` 就是最合适的、官方提供的标准工具。

# 算法笔记：位运算 (Bit Manipulation)

位运算是在数字的二进制表示上进行的操作。它功能强大、效率极高，是优化算法性能、解决特定问题的利器。但其操作不甚直观，需要通过专门的练习来掌握核心技巧和模式。

## 一、 常用位运算符

| 运算符 | 名称 | 描述 | 示例 (`a=5` -> `0101`, `b=3` -> `0011`) |
| :--- | :--- | :--- | :--- |
| `&` | 按位与 (AND) | 两个操作数的对应位都为 1 时，结果位才为 1。 | `a & b` -> `0001` |
| `\|` | 按位或 (OR) | 两个操作数的对应位只要有一个为 1，结果位就为 1。 | `a \| b` -> `0111` |
| `^` | 按位异或 (XOR) | 两个操作数的对应位不同时，结果位为 1。 | `a ^ b` -> `0110` |
| `~` | 按位取反 (NOT) | 将所有 0 变为 1，所有 1 变为 0。（注意 Python 中对负数的处理） | `~a` -> `-6` |
| `<<` | 左移 | 将所有位向左移动指定的位数，右边补 0。相当于乘以 $2^n$。 | `a << 1` -> `1010` |
| `>>` | 右移 | 将所有位向右移动指定的位数，左边（通常）补符号位。相当于除以 $2^n$。 | `a >> 1` -> `0010` |

---

In [1]:
a = 5
~a

-6


## 二、 经典算法题（位运算是核心）

以下是一些面试中的高频题目，使用位运算可以让解法变得极其简洁和高效，甚至是唯一可行的解法。

### 1. 只出现一次的数字 (LeetCode 136)

* **问题描述**: 一个非空整数数组中，除一个元素外，其他元素都出现两次。找出那个只出现一次的元素。
* **核心技巧**: **异或 (XOR) 的性质**。
    1.  `a ^ a = 0` (一个数和它本身异或为 0)
    2.  `a ^ 0 = a` (一个数和 0 异或为它本身)
    3.  异或满足交换律和结合律。
* **解法**: 将数组中所有数字全部异或起来，成对的数字会相互抵消变为 0，最终结果就是那个只出现一次的数字。
* **优势**: 空间复杂度为 O(1)，远优于哈希表的 O(n)。

```python
from typing import List
class Solution:
    def singleNumber(self, nums: List[int]) -> int:
        result = 0
        for num in nums:
            result ^= num
        return result
```

### 2. 2的幂 (LeetCode 231)

* **问题描述**: 判断一个整数 `n` 是否是 2 的幂。
* **核心技巧**: **`n & (n - 1)`**。一个数如果是 2 的幂，其二进制表示中必然只有一个 `1` (如 `8` -> `1000`)。`n - 1` 会得到一个所有相关位都与 `n` 相反的数 (如 `7` -> `0111`)。因此，`n & (n - 1)` 的结果必然为 0。
* **解法**: 判断 `n > 0` 并且 `(n & (n - 1)) == 0`。
* **优势**: O(1) 的时间复杂度，代码极致简洁。

```python
class Solution:
    def isPowerOfTwo(self, n: int) -> bool:
        return n > 0 and (n & (n - 1)) == 0
```

### 3. 两整数之和 (LeetCode 371)

* **问题描述**: **不使用**运算符 `+` 和 `-`，计算两整数 `a` 和 `b` 之和。
* **核心技巧**: **用位运算模拟加法**。
    1.  **无进位和**: `a ^ b` (异或可以计算出不考虑进位的加法结果)。
    2.  **进位**: `(a & b) << 1` (与操作可以找出所有需要进位的位，然后左移一位得到进位值)。
    3.  **迭代**: 将“无进位和”与“进位”重复相加，直到进位为 0。
* **解法**: 循环处理进位，直到进位消失。
* **优势**: 这是题目的**唯一解法**，直接考察对二进制加法原理的理解。

```python
class Solution:
    def getSum(self, a: int, b: int) -> int:
        # Python 的整数是无限精度的，需要处理负数和溢出
        # 32 位掩码
        mask = 0xFFFFFFFF
        
        while b != 0:
            # 计算无进位和与进位
            sum_without_carry = (a ^ b) & mask
            carry = ((a & b) << 1) & mask
            
            a = sum_without_carry
            b = carry
        
        # 如果结果是负数，需要进行转换
        return a if a <= 0x7FFFFFFF else ~(a ^ mask)
```

### 4. 汉明距离 (LeetCode 461)

* **问题描述**: 计算两个整数 `x` 和 `y` 二进制表示中对应位置上数字不同的位数。
* **核心技巧**: **异或(XOR) + 统计比特数**。`x ^ y` 的结果中，为 '1' 的位正好是 `x` 和 `y` 原来不同的位。问题转化为计算 `x ^ y` 中 '1' 的个数。
* **解法**: `(x ^ y).bit_count()` (Python 3.10+)。
* **优势**: 语义和操作完美契合，代码直观且高效。

```python
class Solution:
    def hammingDistance(self, x: int, y: int) -> int:
        return (x ^ y).bit_count()
```

### 5. 缺失的数字 (LeetCode 268)

* **问题描述**: 给定一个包含 `n` 个不同数的数组 `nums`，取自 `0, 1, 2, ..., n`，找出那个未出现在数组中的数字。
* **核心技巧**: **异或 (XOR) 的抵消性质**。将数组中所有数，与 `0` 到 `n` 的完整序列所有数，全部异或在一起。成对出现的数字都会抵消，最终剩下的就是那个缺失的数字。
* **解法**: 一个变量 `missing` 初始化为 `n`，然后与数组的 `索引` 和 `值` 依次异或。
* **优势**: O(1) 的空间复杂度，且避免了求和可能导致的整数溢出问题。

```python
class Solution:
    def missingNumber(self, nums: List[int]) -> int:
        missing = len(nums)
        for i, num in enumerate(nums):
            missing ^= i ^ num
        return missing
```

### 6. 子集 (LeetCode 78)

* **问题描述**: 返回一个数组所有可能的子集（幂集）。
* **核心技巧**: **位掩码 (Bitmask)**。一个长度为 `n` 的数组，其子集有 $2^n$ 个。可以用一个 `n` 位的二进制数（从 `0` 到 $2^n-1$）来与子集一一对应。二进制的第 `j` 位为 `1` 表示选中 `nums[j]`，为 `0` 则不选。
* **解法**: 遍历 `0` 到 $2^n-1$，将每个数字作为掩码来构建子集。
* **优势**: 提供了一种相对于递归回溯而言，更简洁的迭代式生成子集的方法。

```python
class Solution:
    def subsets(self, nums: List[int]) -> List[List[int]]:
        n = len(nums)
        result = []
        # 1 << n 等于 2^n
        for i in range(1 << n):
            subset = []
            for j in range(n):
                # 检查掩码 i 的第 j 位是否为 1
                if (i >> j) & 1:
                # 或 if (1 << j) & i:
                    subset.append(nums[j])
            result.append(subset)
        return result
```

In [ ]:
from typing import List

class Solution1:
    def sortByBits(self, arr: List[int]) -> List[int]:
        
        def count_bits(num: int) -> tuple:
            original_num = num
            count = 0
            while num > 0:
                num = num & (num - 1)
                count += 1
            return (count, original_num)

        return sorted(arr, key=count_bits)

In [ ]:
class Solution2:
    def sortByBits(self, arr: List[int]) -> List[int]:
        
        def count_bits(num: int) -> tuple:
            original_num = num
            count = 0
            while num > 0:
                if num % 2 == 1:  # 检查最低位是否为 1
                    count += 1
                num = num // 2 
            return (count, original_num)

        return sorted(arr, key=count_bits)

In [ ]:
from functools import cmp_to_key

class Solution3:
    def sortByBits(self, arr: List[int]) -> List[int]:

        def compare(a: int, b: int) -> int:
            bit_count_a = a.bit_count()
            bit_count_b = b.bit_count()
            if bit_count_a < bit_count_b:
                return -1
            if bit_count_a > bit_count_b:
                return 1
            if a < b:
                return -1
            if a > b:
                return 1
            return 0

        return sorted(arr, key=cmp_to_key(compare))

In [ ]:
class Solution4:
    def sortByBits(self, arr: List[int]) -> List[int]:
        return sorted(arr, key=lambda num: (num.bit_count(), num))

In [ ]:
class Solution5:
    def sortByBits(self, arr: List[int]) -> List[int]:
        return sorted(arr, key=lambda num: (bin(num).count("1"), num))

# 深入理解计算机整数加法：从 LeetCode 371 出发

这篇笔记以 LeetCode 371 “两整数之和” 为契机，系统性地梳理现代计算机中整数的表示、运算机制，以及硬件和软件实现加法的不同路径。

### 第一部分：数在内存中的组织方式

计算机内存只存储 0 和 1。我们赋予这些二进制位不同的解读方式，来表示不同类型的数字。

#### 1. 无符号数 (Unsigned Integers)

-   **用途**：表示非负整数（0 及正整数）。
-   **存储**：所有位（例如 32 位）都用于存储数值本身。它存储的是数字的**原码**，其二进制表示直接对应十进制值。
-   **示例 (8位)**：`11111111` 表示 `255`。

#### 2. 有符号数 (Signed Integers)

-   **用途**：表示正、负、零整数。
-   **存储**：最高位（Most Significant Bit, MSB）作为**符号位**（`0`为正，`1`为负）。其余位和符号位一起，构成数字的**补码 (Two's Complement)**。
-   **核心机制**：在 C++ 等语言中，当你定义一个 `int a = -5;` 时，内存中存储的不是 `-5` 的原码或反码，而是它的 **32 位补码**。
-   **示例 (8位)**：`11111111` 表示 `-1`。

### 第二部分：整数运算的统一机制

现代计算机最优雅的设计之一，就是将所有整数的加减法，在硬件层面统一为了**补码加法**。

1.  **加法 `a + b`**：直接将 `a` 和 `b` 在内存中的补码，按照二进制规则相加。

2.  **减法 `a - b`**：被转换为 `a + (-b)`。硬件会首先取得 `-b` 的补码，然后与 `a` 的补码进行加法运算。
    -   **如何得到 `-b` 的补码？**：将 `b` 的补码**所有位取反，再加一**。
    -   **数学一致性**：这个操作是其自身的逆操作。
        -   对一个正数 `B` 的补码操作，得到 `-B` 的补码。
        -   对一个负数 `-B` 的补码操作，得到其绝对值 `B` 的补码。
    这两个法则是完全一致的，因为“取反加一”操作本质上是一个正负切换器。

### 第三部分：进位、溢出与正确性保证

-   **最高位进位 (Carry-out)**：从最高有效位（如第 31 位）产生的进位。在补码运算中，这个进位**直接被丢弃**，不影响结果的正确性。

-   **溢出 (Overflow)**：是一种逻辑错误，代表结果超出了当前数据类型（如 32 位 `int`）的表示范围。
    -   **判断条件**：`正+正=负` 或 `负+负=正`。
    -   **实践意义**：在绝大多数应用开发中，溢出是必须**预防**的严重 Bug。通常通过使用更大的数据类型（如 `long long`）来避免。但在一些算法题中，处理溢出本身就是一个考点。

-   **正确性保证**：只要没有发生**溢出**，两个补码相加后，即使产生了最高位进位，丢弃该进位后保留的 32 位结果，**在数学上保证是正确和的补码**。

### 第四部分：本题 C++ 解法的核心任务

基于以上原理，我们可以明确 LeetCode 371 在 C++ 环境下的任务：
1.  我们接收的 `int a` 和 `int b` 参数，在内存中**已经是 32 位的补码**。
2.  题目的输入范围保证了计算**不会发生溢出**。
3.  因此，我们只需要用位运算来**模拟两个 32 位二进制补码的加法过程**。
4.  最终得到的 32 位二进制结果，就是答案的补码。我们直接返回这个 `int` 即可，C++ 语言体系会**自动**将其正确地解读为对应的十进制数。

### 第五部分：硬件 vs. 软件：加法的不同实现路径

我们的算法并非对 CPU 硬件加法器的直接模仿。

#### 1. 逻辑基础：加法器单元

所有二进制加法都基于相同的逻辑单元。

-   **半加器 (Half Adder)**：计算两位 `A` 和 `B`。
    -   和：`Sum = A ^ B`
    -   进位：`Carry = A & B`
-   **全加器 (Full Adder)**：计算三位 `A`、`B` 和前一位的进位 `Cin`。
    -   和：`Sum = A ^ B ^ Cin`
    -   进位：`Cout = (A & B) | ((A ^ B) & Cin)`

#### 2. 硬件实现：追求速度的并行结构

-   **逐位进位加法器 (Ripple-Carry Adder)**：将 32 个全加器串联。逻辑简单，但速度极慢，因为每一位都必须等待前一位的进位结果，产生巨大的“传播延迟”。**现代 CPU 不会使用这种原始设计**。
-   **超前进位加法器 (Carry-Lookahead Adder, CLA)**：现代 CPU 的选择。它通过一套复杂的组合逻辑电路，根据最原始的输入 `A` 和 `B`，**并行地“预测”出所有位的进位**。这打破了进位的串行依赖链，用**增加硬件的复杂度（空间）**换来了**极大的速度提升（时间）**。

#### 3. LeetCode 算法：巧妙的软件迭代

我们的算法是一种**软件层面**的实现，它利用了 CPU 提供的、本身就是并行执行的位运算指令。

-   `sum_without_carry = a ^ b;`：**并行**计算出所有 32 位的“无进位和”。
-   `carry = (a & b) << 1;`：**并行**计算出所有可能产生的进位，并打包成一个新数。
-   `while` 循环：通过**迭代**，将“进位”这个数不断地加到“和”上，这个过程宏观上模拟了进位的传播与吸收，直到不再有任何进位为止。

### 第六部分：C++ 代码实现

```c++
class Solution {
public:
    int getSum(int a, int b) {
        // 当 b（用于存储进位）不为 0 时，循环继续。
        while (b != 0) {
            // 1. 计算进位 (Carry)。
            // a & b 找到所有需要进位的 bit。
            // 使用 unsigned int 来进行左移，可以避免在 C++ 中对负数
            // （最高位为1）进行左移时的未定义行为，这是一种更安全严谨的做法。
            unsigned int carry = static_cast<unsigned int>(a & b) << 1;
            
            // 2. 计算无进位和 (Sum without Carry)。
            a = a ^ b;
            
            // 3. 将进位值赋给 b，用于下一次迭代。
            b = carry;
        }
        
        // 循环结束时，所有进位都已被加到 a 中，a 即为最终结果。
        return a;
    }
};
```